In [ ]:
%pip install -r requirements.txt

In [1]:
import polars as pl
import df_gen as dg
df = dg.generate_dummy_order_data(100)

In [9]:
#here's what we need to do to generate a delta table
df.write_delta('./delta_warehouse/orders')

In [ ]:
# heres what we have to do to generate an iceberg table
from pyiceberg.catalog.sql import SqlCatalog
warehouse_path = "./iceberg_warehouse"
catalog = SqlCatalog("default", **{"uri": "sqlite:///:memory:", "warehouse": f"file://{warehouse_path}",})
df_ice = df.to_arrow()
catalog.create_namespace_if_not_exists("dummy_data")
ice_table = catalog.create_table("dummy_data.orders",schema=df_ice.schema)
ice_table.append(df_ice)
# need this to get the latest metadata file to point to
print(ice_table.metadata_location)

In [ ]:
print(ice_table.metadata_location)

In [ ]:
#gcs
import os
bucket = os.getenv("GCS_BUCKET")
project = os.getenv("GBQ_PROJECT_ID")

from pyiceberg.catalog.sql import SqlCatalog
warehouse_path = f"gs://{bucket}/icehouse1"
catalog = SqlCatalog("default",
                      **{"uri": "sqlite:///:memory:",
                         "warehouse": warehouse_path,
                         "gcs.project-id": project,
                        },
                    )
df_ice = df.to_arrow()
catalog.create_namespace_if_not_exists("dummy_data")
ice_table = catalog.create_table("dummy_data.orders",schema=df_ice.schema)
ice_table.append(df_ice)
# need this to get the latest metadata file to point to
print(ice_table.metadata_location)